In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.data import Data, DataLoader

In [2]:
import mlflow
import mlflow.pytorch
from datetime import datetime

In [4]:
# 1. Настройка MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("PyTorch cs2")

KeyboardInterrupt: 

In [2]:
# Проверяем доступность CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:

# Определяем модель
class CS2GNN(nn.Module):
    def __init__(self, num_node_features, hidden_dim=64, heads=4):
        super().__init__()
        self.conv1 = GATConv(num_node_features, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, edge_index, team1_indices, team2_indices):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        team1_features = x[team1_indices].mean(dim=0)
        team2_features = x[team2_indices].mean(dim=0)
        combined = torch.cat([team1_features, team2_features], dim=0)
        return self.fc(combined)

In [ ]:
# Перенос модели на GPU
model = CS2GNN(num_node_features=5).to(device)

In [ ]:
# Генерация синтетических данных (замените на реальные)
def generate_fake_data(num_players=10, num_features=5):
    x = torch.randn(num_players, num_features).to(device)
    team1_edges = [(i, j) for i in range(5) for j in range(5) if i != j]
    team2_edges = [(i+5, j+5) for i in range(5) for j in range(5) if i != j]
    edge_index = torch.tensor(team1_edges + team2_edges, dtype=torch.long).t().contiguous().to(device)
    team1_indices = torch.tensor([0, 1, 2, 3, 4], device=device)
    team2_indices = torch.tensor([5, 6, 7, 8, 9], device=device)
    return Data(x=x, edge_index=edge_index), team1_indices, team2_indices

In [ ]:
# Создаем датасет и DataLoader
dataset = [generate_fake_data()[0] for _ in range(100)]  # 100 фейковых матчей
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss и оптимизатор
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Функция обучения
def train():
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)  # Переносим batch на GPU
        optimizer.zero_grad()
        
        # Генерируем случайные метки для примера (в реальности используйте настоящие)
        y_true = torch.randint(0, 2, (1,)).float().to(device)
        
        # Получаем предсказания
        team1_indices = torch.tensor([0, 1, 2, 3, 4], device=device)
        team2_indices = torch.tensor([5, 6, 7, 8, 9], device=device)
        out = model(batch.x, batch.edge_index, team1_indices, team2_indices)
        
        # Считаем loss и обновляем веса
        loss = criterion(out, y_true)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# Цикл обучения
# 4. Обучение с логированием в MLflow
with mlflow.start_run():
    # Логируем гиперпараметры
    mlflow.log_params({
        "batch_size": 32,
        "learning_rate": 0.001,
        "optimizer": "Adam"
    })
    for epoch in range(1, 101):
        loss = train()
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

In [ ]:
# Функция предсказания
def predict(model, team1_features, team2_features):
    model.eval()
    with torch.no_grad():
        # Собираем граф для двух команд
        x = torch.cat([team1_features, team2_features], dim=0).to(device)
        edge_index = torch.tensor([(i, j) for i in range(5) for j in range(5) if i != j] + 
                                [(i+5, j+5) for i in range(5) for j in range(5) if i != j], 
                                dtype=torch.long).t().contiguous().to(device)
        team1_indices = torch.tensor([0, 1, 2, 3, 4], device=device)
        team2_indices = torch.tensor([5, 6, 7, 8, 9], device=device)
        prob = model(x, edge_index, team1_indices, team2_indices)
    return prob.item()

In [ ]:
# Пример предсказания
team1_features = torch.randn(5, 5).to(device)  # Признаки 5 игроков team1
team2_features = torch.randn(5, 5).to(device)  # Признаки 5 игроков team2
print(f"Probability that team1 wins: {predict(model, team1_features, team2_features):.4f}")